In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import ee
import xee
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
import xarray
import matplotlib.pyplot as plt
from shapely.geometry import box
from scipy.signal import savgol_filter
import rasterio
from rasterio.features import rasterize
from osgeo import gdal
import rioxarray

from func_mask import get_s2, add_cld_shdw_mask, apply_mask, addNDVI
from func_processing import time_series


In [2]:
try:
    #ee.Initialize()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()

# Downloading time series
In this Jupyter notebook, raster time series are downloaded with the Xarray Earth Engine (XEE) package.

## Settings

In [ ]:
# Voor twee jaar
start = '2021-11-01' # iets langer dan period of interest zodat we kunnen interpoleren
end = '2025-01-15'

start_wdw = '2022-02-01' # de echte period of interest
end_wdw = '2024-09-01'

vi_str = 'bsi' # welke vegetatie-index
sg_filter = True # gebruik van een Savitzky-Golay filter 

output_folder = 'Q:/Projects/PRJ_MBAG/4d_bwk/project-telcirkels/rasters-telcirkels-mas'
export = True

## Reading geometries (telcirkels)
Processing of the geometries was done in R script 01-process-geometries.R. Outputs were written to shared Q-drive. These processed geometries are now loaded here, so that they can be used in xarray earth engine. The counting points stay the same over the years, this means that we will download the required imagery for all years we want (e.g. 2022 and 2023) at the same time.

In [7]:
#Read telcirkels
#  CRS to 4326!!

tc = gpd.read_file('../data/processed/telcirkels_viz.gpkg')
tc = tc.to_crs(epsg=4326)
tc.head(5)

,pointid,sample_order,batch,regio,area_prop_sb,openheid_klasse,sbp,crs,geometry
0,Km_10019,1.0,eerste set,Kempen,0.001557,HOL,binnen,31370,"POLYGON ((5.46512 51.10721, 5.46511 51.10707, ..."
1,Km_10056.3,5.0,eerste set,Kempen,0.000000,HOL,binnen,31370,"POLYGON ((5.42861 51.06523, 5.42860 51.06509, ..."
2,Km_10910,57.0,eerste set,Kempen,0.000000,HOL,binnen,31370,"POLYGON ((5.75928 51.16607, 5.75927 51.16593, ..."
3,Km_12411.5,41.0,eerste set,Kempen,0.000000,HOL,binnen,31370,"POLYGON ((4.77799 51.10468, 4.77798 51.10454, ..."
4,Km_13799.2,45.0,eerste set,Kempen,0.000000,HOL,binnen,31370,"POLYGON ((4.92203 51.38162, 4.92203 51.38148, ..."


In [ ]:
## testje
#tc = tc.loc[tc.pointid.isin(['Lm_11070','Lm_11568.1','Lm_15064.9', 'Lm_8486.3', 'Lm_8898.1.8', 'Lm_9709.1.6'])]
#tc = tc.reset_index(drop=True)

## redownload

# Which points should be redownloaded?
#path = output_folder
#files_list = os.listdir(path)

#size_of_file = [
#    (f, os.stat(os.path.join(path, f)).st_size) for f in files_list
#]

# Iterate over list of files along with size 
# and print them one by one.

#tc_to_redownload = [tc[0] for tc in size_of_file if tc[1]<4000000]
#tc_names = [tc.lstrip('multiband_raster_ndvi_').rstrip('_sg.tif') for tc in tc_to_redownload]
#print(tc_names)

#tc = tc.loc[tc.pointid.isin(tc_names), ].reset_index()

KeyboardInterrupt: 

## Processing for-loop
- Getting time series for every counting point, for all years at once
- Exporting them

In [68]:
# Loop over every telcirkel, for each one, data
sg_name = '_sg' if sg_filter == True else ''
failed_downloads = list()

for i in range(tc.shape[0]): #
    # create a bbox and convert to right format for extracting time series
    pointid = tc.loc[i,'pointid']
    shape = tc.loc[i,'geometry']


    # Create time series with custom function
    da = time_series(start, end, start_wdw, end_wdw, shape, vi_arg = vi_str, sg = sg_filter) #input= 'EPSG:4326', output = 'EPSG:32631'
    da.rio.set_spatial_dims('X', 'Y', inplace=True)

    #-- EXPORT
    if export == True:
        # Rename dimensions to conform to rioxarray requirements
        da = da \
        .rename({'Y': 'y', 'X': 'x'}) \
        .transpose('time', 'y', 'x') \
        .rio.write_crs("EPSG:32631")


        # Specify the output file path & export as GTiff

        output_file = f'multiband_raster_{vi_str}_{pointid}{sg_name}.tif'
        output_path = os.path.join(output_folder, output_file)

        try:
            da.rio.to_raster(output_path, driver='GTiff', tiled=True, compress='LZW')

            # Band renaming with GDAL (to store the dates)
            gdal.UseExceptions()

            # Define band names (these should be in the order of your bands)
            band_names = [str(time).split('T')[0] for time in da.time.values]

            # Open the dataset in update mode
            dataset = gdal.Open(output_path, gdal.GA_Update)

            if dataset:
                for i, name in enumerate(band_names, start=1):
                    band = dataset.GetRasterBand(i)
                    if band:
                        band.SetDescription(name)  # Set the band name

                dataset = None  # Close the dataset to flush changes
            else:
                print("Error opening file")
        except:
            print(f'Download failed for pointid {pointid}')
            failed_downloads.append(pointid)

crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631


## Postprocessing
For 30 points there were NA values left in the time series, therefore no savgol function could be applied. So I downloaded these time series separately, without applying the savgol filter. Here, I want to apply some postprocessing (i.e. the same savgol filter applied to all other time series) to these points. 

Goal is to apply to only the time series with no NA values

In [98]:
import xarray as xr
import rioxarray as rxr
from scipy.signal import savgol_filter
import numpy as np

# List functions
def apply_savgol_to_1d(arr, window_length, polyorder):
    """
    Applies the Savitzky-Golay filter to a 1D numpy array,
    ignoring NaN values.
    """
    if np.isnan(arr).all():
        return arr
    
    # Get indices of non-NaN values
    not_nan_indices = ~np.isnan(arr)
    
    # Apply filter to non-NaN values
    filtered_values = savgol_filter(arr[not_nan_indices], window_length, polyorder)
    
    # Create an output array with the original size
    output_arr = np.full_like(arr, np.nan)
    
    # Place filtered values back in their original positions
    output_arr[not_nan_indices] = filtered_values
    
    return output_arr

def savgol_filter_xr(da, dim, window_length, polyorder):
    """
    Applies the Savitzky-Golay filter to an xarray.DataArray along a specified dimension,
    handling NaN values.
    """
    return xr.apply_ufunc(
        apply_savgol_to_1d,
        da,
        input_core_dims=[[dim]],
        output_core_dims=[[dim]],
        exclude_dims={dim},
        kwargs={"window_length": window_length, "polyorder": polyorder},
        dask='parallelized',
        output_dtypes=[da.dtype],
        vectorize=True,
        keep_attrs=True
    )


window_length = 5
polyorder = 2

#-- Apply functions
# List all files that don't end with '_sg' (=savgol)
rasters = [file for file in os.listdir(output_folder) if not file.endswith('_sg.tif')]

for raster_name in rasters:

    raster_arr = rxr.open_rasterio(os.path.join(output_folder, raster_name))

    # Apply the savgol filter
    filtered_raster = savgol_filter_xr(raster_arr, 'band', window_length, polyorder)

    # Save the filtered raster
    filtered_raster.transpose('band', 'y', 'x').rio.to_raster(os.path.join(output_folder, raster_name.rstrip('.tif') + '_sg.tif'))